In [ ]:
import pickle
import cobra

In [ ]:
# Load the results
with open('results.pkl', 'rb') as f:
    experiment = pickle.load(f)

In [ ]:
# Load the COBRA model
alt_model = cobra.io.read_sbml_model("../../../../GEM-repos/mit1002-model/model.xml")

In [ ]:
ecoli_model = cobra.io.load_json_model("../../../../GEM-repos/ecoli/iJO1366.json")

In [ ]:
def get_readable_metabolite_name(metabolite):
    return f"{metabolite.name} ({metabolite.id})"

In [ ]:
#metabolite_id = "cpd00007_c0" # (O2)
# metabolite_id = "cpd00011_c0" # (CO2)
metabolite_id = "cpd00002_c0" # (ATP)

In [ ]:
# Check if the metabolite exists in the model
if metabolite_id in alt_model.metabolites:
    metabolite = alt_model.metabolites.get_by_id(metabolite_id)
else:
    raise ValueError(f"Metabolite {metabolite_id} not found in the model.")

In [ ]:
# Find all reactions containing the metabolite
reactions_with_metabolite = [reaction for reaction in alt_model.reactions if metabolite in reaction.metabolites]

In [ ]:
reactions_with_metabolite

In [ ]:
# Get the fluxes for a specific cycle (the first, index 0) of the simulation
fluxes = experiment.fluxes_by_species[''].iloc[0]

In [ ]:
# Print the nonzero flux reactions containing the metabolite, flux, and bounds
for reaction in reactions_with_metabolite:
    reaction_id = reaction.id
    flux = fluxes[reaction_id]
    
    # Filter out reactions with near-zero flux (using a threshold, e.g., 1e-6)
    if abs(flux) > 1e-6:
        lower_bound = reaction.lower_bound
        upper_bound = reaction.upper_bound
        readable_reaction_string = " + ".join([f"{coefficient} {get_readable_metabolite_name(met)}" for met, coefficient in reaction.metabolites.items()])
        print(f"{reaction_id} | {reaction.name} | {readable_reaction_string} : {flux} | Bounds: [{lower_bound}, {upper_bound}]")

In [ ]:
# Print the nonzero flux reactions containing the metabolite, flux, and bounds
for reaction in reactions_with_metabolite:
    reaction_id = reaction.id
    flux = fluxes[reaction_id]
    
    # Filter out reactions with near-zero flux (using a threshold, e.g., 1e-6)
    if abs(flux) < 1e-6:
        continue
    
    # Get reactions where CO2 is being used (i.e. the it is a product and the 
    # flux is negative or it is a reactant and the flux is positive)
    if (metabolite in reaction.products and flux > 0) or (metabolite in reaction.reactants and flux < 0):
        lower_bound = reaction.lower_bound
        upper_bound = reaction.upper_bound
        readable_reaction_string = " + ".join([f"{coefficient} {get_readable_metabolite_name(met)}" for met, coefficient in reaction.metabolites.items()])
        print(f"{reaction_id} | {reaction.name} | {readable_reaction_string} : {flux} | Bounds: [{lower_bound}, {upper_bound}]")

In [ ]:
# Get all the reactions that can produce ATP
metabolite_id = "cpd00002_c0" # (ATP)
metabolite = alt_model.metabolites.get_by_id(metabolite_id)

for reaction in alt_model.reactions:
    if metabolite in reaction.products:
        if 'bigg.reaction' in reaction.annotation.keys():
            bigg_id = reaction.annotation['bigg.reaction']
        else:
            bigg_id = 'No BiGG ID'
        lower_bound = reaction.lower_bound
        upper_bound = reaction.upper_bound
        readable_reaction_string = " + ".join([f"{coefficient} {get_readable_metabolite_name(met)}" for met, coefficient in reaction.metabolites.items()])
        print(f"{reaction.id} | {bigg_id} | Bounds: [{lower_bound}, {upper_bound}] | {reaction.name} | {readable_reaction_string}")
    if metabolite in reaction.reactants and reaction.lower_bound < 0:
        if 'bigg.reaction' in reaction.annotation.keys():
            bigg_id = reaction.annotation['bigg.reaction']
        else:
            bigg_id = 'No BiGG ID'
        lower_bound = reaction.lower_bound
        upper_bound = reaction.upper_bound
        readable_reaction_string = " + ".join([f"{coefficient} {get_readable_metabolite_name(met)}" for met, coefficient in reaction.metabolites.items()])
        print(f"{reaction.id} | {bigg_id} | Bounds: [{lower_bound}, {upper_bound}] | {reaction.name} | {readable_reaction_string}")

In [ ]:
for reaction in alt_model.reactions:
    if (metabolite in reaction.products) or (metabolite in reaction.reactants and reaction.lower_bound < 0):
        if 'bigg.reaction' in reaction.annotation.keys():
            bigg_id = reaction.annotation['bigg.reaction']
            ec_reaction = ecoli_model.reactions.get_by_id(bigg_id)
            ec_lower_bound = ec_reaction.lower_bound
            ec_upper_bound = ec_reaction.upper_bound
        else:
            bigg_id = 'No BiGG ID'
        lower_bound = reaction.lower_bound
        upper_bound = reaction.upper_bound
        readable_reaction_string = " + ".join([f"{coefficient} {get_readable_metabolite_name(met)}" for met, coefficient in reaction.metabolites.items()])
        # Look up the reaction in the E. coli model
        for id in b
        print(f"{reaction.id} / {bigg_id}/ {reaction.name}")
        print("----------------------------------")
        print(f"Bounds in ALT: [{lower_bound}, {upper_bound}]")
        print(f"Bounds in E. coli: [{ec_lower_bound}, {ec_upper_bound}]")
        print(f"{readable_reaction_string}\n")